[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1_jWs5c-dG8auYbPgYP-harE9qe35Vvh5?usp=sharing)

# TF2 Object detection API

This notebook is a simple implementation of the TF2 Object Detection API. 
It is designed to work out-of-the-box inside google colab. 

The use of the Colab is suggested when the intention is to deploy a simple detection application quickly. In the case of more complex solutions, it is suggested to train on-premises machines or local GPU machines.

This Colab Notebook will:

* Install TensorFlow2 Object Detection API and all the required dependencies
* Split initial dataset, create label_map and tensor flow record files
* Create configuration required configurations and load pre-trained models
* Retrain a model to adapt custom models to specific detection classes
* Export custom trained model
* Perform inference by using the model trained

# Environment setting up

- Connect the notebook to google drive to access training and test dataset
- Install Tensorflow 2, TF2 Object Detection API and all the required dependancies
- Run TF2 Object Detection API tests to validate the installation

### Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Install TensorFlow2 and the Object Detection API

The installation pull the required files from the repository: https://github.com/tensorflow/models


In [ ]:
import os
import pathlib
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET


# Clone the tensorflow models repository
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# load all the required libraries and dependancies
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
#run model builder test
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py


# Split initial dataset, create label_map and tensor flow record files.

This section will implement some helper functions to produce the required files that will be provided afterwards to the TF2 Object Detection API

The steps here performed are:

* Set some general information about paths and folders
* Perform Train and Test set splitting
* Create the required *label_map.pbtxt*
* Convert all the information into .record

### General paramenters and helper functions loading

In [ ]:
MAIN_PATH = "/content/drive/My Drive/Colab Notebooks/QR_detector/application/" # main path where all the required folder are reported
IMAGE = "image" # name of the folder that contains the images 
ANNOTATIONS = "annotation" # name of the folder that contains annotation .xml in Pascal VOC format

In [ ]:
# Training paramenters
TEST_SIZE = 0.20 # ratio of the test dataset > 20% of the entire images
SPLIT_SEED = 42 # this allow to reproduce the split several times

In [ ]:
# Clone git repo with helper function to generate TF_record
if "TF2_object_detection_api" in pathlib.Path.cwd().parts:
  while "TF2_object_detection_api" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('TF2_object_detection_api').exists():
  !git clone https://github.com/FrancescoMrn/TF2_object_detection_api.git

In [ ]:
#create images paths and annotation 
image_dir = os.path.join(MAIN_PATH,IMAGE)
annotation_dir = os.path.join(MAIN_PATH,ANNOTATIONS)

### Perform Train and Test split

In [ ]:
# check images and annotation
images = [x.split(".")[0] for x in os.listdir(image_dir)]
X_train, X_test = train_test_split(images, test_size=TEST_SIZE, random_state=SPLIT_SEED)

In [ ]:
def move_data(data, src_dir, output_path, folder="train"):
    for i in data:
        img_i = i+".jpg"
        ann_i = i+".xml"
        
        # check if the output folder is present 
        if not os.path.exists(os.path.join(output_path, folder)):
          os.makedirs(os.path.join(output_path, folder))

        # move files into train and test
        shutil.copyfile(os.path.join(src_dir[0], img_i), os.path.join(output_path, folder, img_i))
        shutil.copyfile(os.path.join(src_dir[1], ann_i), os.path.join(output_path, folder, ann_i))
    print("All files have been moved to {}".format(output_path+folder))

In [ ]:
move_data(X_train, [image_dir, annotation_dir], output_path=MAIN_PATH, folder="train")
move_data(X_test, [image_dir, annotation_dir], output_path=MAIN_PATH, folder="test")

In [ ]:
# create path for TF record and annotation to perform conversion in .record
path_tfrecords_train = os.path.join(MAIN_PATH, "train.record").replace(" ","\ ")
path_tfrecords_test = os.path.join(MAIN_PATH, "test.record").replace(" ","\ ")
path_annotation_train = os.path.join(MAIN_PATH, "train").replace(" ","\ ")
path_annotation_test = os.path.join(MAIN_PATH, "test").replace(" ","\ ")

### Create label map

In [ ]:
path_label_map = os.path.join(MAIN_PATH, "label_map.pbtxt").replace(" ","\ ")
path_label_map

To create the label_map.pbtxt in the right format the script below is provided. 
The label map format is the following
```
item {
  id: 1
  name: 'nine'
}

item {
  id: 2
  name: 'ten'
}

item {
  id: 3
  name: 'jack'
}
```

Each class that is has an ID and the corrisponding name - **the same used during the labeling**

**Note** copy the path from above */content/drive/My Drive/path to the label map/label_map.pbtxt* in the bash script below

In [ ]:
%%bash 
cat <<EOT >> /content/drive/My\ Drive/Colab\ Notebooks/QR_detector/application/label_map.pbtxt
item {
  id: 1
  name: 'QR'
}
EOT

In [ ]:
!python /content/TF2_object_detection_api/generate_tfrecord_V2.py\
        -x $path_annotation_train \
        -l $path_label_map \
        -o $path_tfrecords_train

In [ ]:
!python /content/TF2_object_detection_api/generate_tfrecord_V2.py \
        -x $path_annotation_test \
        -l $path_label_map \
        -o $path_tfrecords_test

# Custom TF2 Object Detection training configuration

In this section will be setup the paths to train, test. These two files are required to be .record file. Additionally in this section is required to provide the path to the label map (.pbtxt). 

Finally the it is required to select the model. In this notebook only some of the most peformant models are reported. Check it out the entire list of models here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


- Provide path to the following files: train.record, test.record and label_map.pbtxt
- Select the model that will be later trained on the dataset provided
- Create the required configuration files

### Provide required path to train, test and label map

In [ ]:
# train_record_fname / test_record_fname: format "/content/drive/My Drive/<path to the file>/train.record"
train_record_fname = path_tfrecords_train.replace("\ "," ")
test_record_fname = path_tfrecords_test.replace("\ "," ")
# label_map_pbtxt_fname: format "/content/drive/My Drive/<path to the file>/label_map.pbtxt"
label_map_pbtxt_fname = path_label_map.replace("\ "," ") #"/content/drive/My Drive/Colab Notebooks/QR_detector/development/label_map.pbtxt"

### Select pretrained model

In [ ]:
# Short-list of models - to discover the entire list of model check tf2_detection_zoo
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 8
    }
}

In [ ]:
# model training parameters
MODEL_SELECTED = 'efficientdet-d0' # select your model from MODEL CONFIG dictionary
NUM_STEPS = 4000 # The more steps, the longer the training.
NUM_EVAL_STEPS = 500 # #Perform evaluation after so many steps


model_name = MODELS_CONFIG[MODEL_SELECTED]['model_name']
pretrained_checkpoint = MODELS_CONFIG[MODEL_SELECTED]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[MODEL_SELECTED]['base_pipeline_file']
batch_size = MODELS_CONFIG[MODEL_SELECTED]['batch_size']

In [ ]:
# download pretrained weights
%mkdir /content/models/research/deploy/
%cd /content/models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

In [ ]:
# download base training configuration file
%cd /content/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
#prepare
pipeline_fname = '/content/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
    
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
#write custom configuration file
import re

%cd /content/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(NUM_STEPS), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)



In [ ]:
# path where the pipeline file is saved 
pipeline_file = '/content/models/research/deploy/pipeline_file.config'
# path where the model training will be saved
model_dir = '/content/training/'

# Train Custom TF2 Object Detector

In this section the custom model will be trained. 
The parameters required for training are the following:

* pipeline_file: defined above in writing custom training configuration
* model_dir: path of tensorboard logs and saved model checkpoints
* num_train_steps: number of steps before stopping the training
* num_eval_steps: eval on validation set after this many steps

*Note: in Google Colab the run is terminated automatically after 12 hrs. Be sure to select Runtime-Change runtime-GPU to speedup the training process.*






In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={NUM_STEPS} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={NUM_EVAL_STEPS}

### Tensorboard Logs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

# Export custom trained model

This section will export the inference graph for later use. 

*Note: This section is still work in progress*

In [ ]:
# check model weights folder 
%ls '/content/training/'

In [ ]:
#run conversion script
import re
import numpy as np
output_directory = os.path.join(MAIN_PATH,"fine_tuned_model").replace(" ","\ ") # this is the folder where the inference graph will be saved
#output_directory = '/content/fine_tuned_model' # this is the folder where the inference graph will be saved

# model weights exporter
print("Folder where the model is loaded:",model_dir)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

# Perform inference by using the model trained

This last section perfom model inference over the test dataset. We are going to use the following files created durign the configuration and training of the model:

- pipeline_file: contains all the manipulation of the input files
- model training checkpoint: is the file containing the checkpoint saved during the training 
- test images path: path to the images to be tested



*This section will be updated with the graph inference for better peformances*

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
%ls '/content/training/'

In [ ]:
# saved pipeline file
pipeline_config = pipeline_file

# generally you want to put the last ckpt from training
model_dir = '/content/training/ckpt-5'

In [ ]:
def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

In [ ]:
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(model_dir))

# get model function
detect_fn = get_model_detection_function(detection_model)

In [ ]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
#run detector
import random

# TEST_IMAGE_PATHS format: "/content/drive/My Drive/<path to the folder>/*.jpg"
TEST_IMAGE_PATHS = glob.glob(os.path.join(MAIN_PATH,"test/*.jpg"))
image_path = random.choice(TEST_IMAGE_PATHS)
image_np = load_image_into_numpy_array(image_path)

# Small data manipulation
# Flip horizontally
# image_np = np.fliplr(image_np).copy()
# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.5,
      agnostic_mode=False,
)

plt.figure(figsize=(12,12))
plt.imshow(image_np_with_detections)
plt.show()